# Introduction to Text Mining

-----

In this notebook, we explore more advanced machine learning techniques with text data. First, we introduce the concept of n-grams, which are combinations of one or more tokens. For example, bigrams are combinations of two tokens, while trigrams are combinations of three tokens. Next, we introduce the concept of stemming, which is used to convert tokens into their root forms so that token frequencies match the use of the root token rather than being spread across multiple similar tokens. Finally, we explore the application of clustering and feature selection to text data.

-----

## Table of Contents

[N-Grams](#N-Grams)

[N-Gram Classification](#N-Gram-Classification)

[Stemming](#Stemming)

[Clustering Analysis](#Clustering-Analysis)

[Dimension-Reduction](#Dimension-Reduction)

-----

Before proceeding with the rest of this notebook, we first include the notebook setup code and we define our _home_ directory.

-----

In [1]:
# Set up Notebook
% matplotlib inline

# Standard imports
import numpy as np

In [2]:
# First we find our HOME directory
home_dir = !echo $HOME

# Then we create a temporary working directory to store data files in
!mkdir ~/temp_w9l3

# We construct the full path, below our   
# HOME directory to the temporary working directory location
home=home_dir[0] + '/temp_w9l3/'

-----

[[Back to TOC]](#Table-of-Contents)

## N-Grams

Formally, a [_n-gram_][ngd] is a contiguous sequence of **n** items from a parent sequence of items, such as characters or words in a text document. In general, we will focus solely on words in a document. Thus, our initial approach has simply been to look at unigrams or single words in a document when building a classification model. However, sometimes the combination of words can be more descriptive, for example, _unbelievably bad_ is generally viewed as a more powerful description than just _bad_. As a result, the concept of an _n-gram_ was created, where collections of words can be treated as features. In fact google allows a user to search for [specific n-gram][gnv] combinations in books that they have scanned.

While this clearly can improve classification power, it also increases computational requirements. This is a result of the exponential rise in the number of possible features. For example, given $n$ words, we have $n \times (n - 1)$ possible bigrams, and so on for higher order combinations. While this is not a problem for small vocabularies, for larger vocabularies (and corresponding documents) the number of possible features can quickly become very large. Thus, many text mining applications will make use of Hadoop or Spark clusters to leverage the inherent parallelism in these tasks.

To demonstrate using n-grams, we first demonstrate the concept on a single sentence.

-----
[gnv]: https://books.google.com/ngrams
[ngd]: https://en.wikipedia.org/wiki/N-gram

In [3]:
my_text = 'This course introduces many concepts in data science.'

# Tokenize sentance
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,3))

# Analyze sentance
tk_func = cv.build_analyzer()

# Display n-grams
import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, 
                          width=80, compact=True)
pp.pprint(tk_func(my_text))

[ 'this', 'course', 'introduces', 'many', 'concepts', 'in', 'data', 'science',
  'this course', 'course introduces', 'introduces many', 'many concepts',
  'concepts in', 'in data', 'data science', 'this course introduces',
  'course introduces many', 'introduces many concepts', 'many concepts in',
  'concepts in data', 'in data science']


-----

We can create a new document-term matrix based on this sentence, and use this matrix (or vector since it is only one row), to provide a representation space for new sentences. In the following Code cell, we tokenize our original sentence, and sort the resulting vocabulary (i.e., n-grams) with their ranking (or order). Next, we create a second, simple sentence and compute the indices into the original DTM for the n-grams in the new sentence. The result is display as a bit vector, where `1` means the corresponding n-gram is in the new sentence, and `0` means it is not.


-----

In [4]:
# Display token mapping
in_list = []
in_list.append(my_text)

# Tokenize sentence
cv = cv.fit(in_list)

# Sort tokens
import operator
my_voc = sorted(cv.vocabulary_.items(), key=operator.itemgetter(1))

# Display token mapping
print('Token mapping:')
print(40*'-')

for tokens, rank in my_voc:
    print(rank, tokens)

# Display new sentence
print(40*'-')
out_list = ['This course is data science!']

# Transform new sentence to original sentence DTM
xsm = cv.transform(out_list)
print(out_list)

# Display count vector indices for new sentance tokens
print(40*'-')
print(xsm.todense())

Token mapping:
----------------------------------------
0 concepts
1 concepts in
2 concepts in data
3 course
4 course introduces
5 course introduces many
6 data
7 data science
8 in
9 in data
10 in data science
11 introduces
12 introduces many
13 introduces many concepts
14 many
15 many concepts
16 many concepts in
17 science
18 this
19 this course
20 this course introduces
----------------------------------------
['This course is data science!']
----------------------------------------
[[0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0]]


-----

<font color='red' size = '5'> Student Exercise </font>

In the preceding cells, we used `CountVectorizer` to create n-gram tokens. Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Change the `CountVectorizer` to use stop words and change the tokens to all lowercase. How does this change the tokens and mappings? 
2. Create your own sentence, do the tokens and n-grams make sense?
3. Try changing the ngram range to different values (e.g., `ngram_range=(1,4)` or `ngram_range=(2,3)`). Notice how the number of tokens quickly increase.

-----

[[Back to TOC]](#Table-of-Contents)

## N-Gram Classification

Having n-grams offers improved classification, since word or token combinations often include more information than single words or tokens. For example, _University Illinois_ means more than just _University_ and _Illinois_. We can build on our previous simple text classification pipeline to now develop a more complete code example that builds a feature vector containing both single words and b-grams from the documents. We use this new sparse matrix to classify the documents by using our simple Naive Bayes classifier, which obtains slightly better results. First, we load the movie review data.

-----

In [5]:
# Load movie corpus
import nltk
mvr = nltk.corpus.movie_reviews

# Use scikit learn to split into training and testing
from sklearn.datasets import load_files
data_dir = '/home/data_scientist/data/nltk_data/corpora/movie_reviews'

mvr = load_files(data_dir, shuffle = False)
print('Number of Reviews: {0}'.format(len(mvr.data)))

from sklearn.model_selection import train_test_split
mvr_train, mvr_test, y_train, y_test = train_test_split(
    mvr.data, mvr.target, test_size=0.25, random_state=23)

Number of Reviews: 2000


In [6]:
# Naive Bayes pipeline to classify
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

tools = [('cv', CountVectorizer()), ('nb', MultinomialNB())]
pclf = Pipeline(tools)

# Lowercase and restrict ourselves to about 
# half the available features
pclf.set_params(cv__stop_words = 'english',
                cv__ngram_range=(1,2),
                cv__lowercase=True)

# Fit model, predict, and display results
pclf.fit(mvr_train, y_train)
y_pred = pclf.predict(mvr_test)
print(metrics.classification_report(y_test, y_pred, 
                                    target_names = mvr.target_names))

             precision    recall  f1-score   support

        neg       0.84      0.71      0.77       259
        pos       0.73      0.85      0.79       241

avg / total       0.79      0.78      0.78       500



In [7]:
# Extract the classifier
clf = pclf.steps[1][1]

# Display number of features
print(f'Number of Features = {clf.feature_log_prob_.shape[1]}')

Number of Features = 420997


-----

Now we can repeat the results, but now use unigrams, bigrams, and trigrams. Since this will produce a document term matrix that likely exceeds the computational resources of our Docker container (i.e., a very large DTM can crash your server), we impose two cuts on the tokens included in our DTM. First, we impose a minimum feature term that requires a term to be present in at least two documents. Second, we set a maximum frequency of 50%, such that any term occurring in more than fifty percent of all documents will be ignored (these are likely all stop words, but this value can be reduced to a lower value to trim frequently occurring words that add little to the accuracy. Notice that our overall number of features is much smaller, even though we are also using trigrams in this classification pipeline.

-----

In [8]:
# Restrict vocabulary, even with trigrams
pclf.set_params(cv__stop_words = 'english',
                cv__ngram_range=(1,3),
                cv__lowercase=True,
                cv__min_df=2,
                cv__max_df=0.5)

# Fit, predict, and display results
pclf.fit(mvr_train, y_train)
y_pred = pclf.predict(mvr_test)
print(metrics.classification_report(y_test, y_pred, 
                                    target_names = mvr.target_names))

             precision    recall  f1-score   support

        neg       0.84      0.78      0.81       259
        pos       0.78      0.83      0.81       241

avg / total       0.81      0.81      0.81       500



In [9]:
# Extract the classifier
clf = pclf.steps[1][1]

# Display number of features
print(f'Number of Features = {clf.feature_log_prob_.shape[1]}')

Number of Features = 62734


-----

<font color='red' size = '5'> Student Exercise </font>

In the preceding cells, we included n-grams in the classification process by using a `CountVectorizer`.  Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Try to determine a value of `max_df` that replicates the effects of using stop words. 
2. Change `CountVectorizer` to `TfidfVectorizer`. Do the results change?
3. Try using a more powerful classification algorithm, as opposed to `MultinomialNB`. Do the results change?

-----

[[Back to TOC]](#Table-of-Contents)

## Stemming

So far, we have looked at several techniques to remove redundant or unimportant features. For example, we changed the case of all text to lowercase and we have applied stop words. However, there still is the issue of different forms of the same word, for example compute, computer, computed, and computing. The process of changing words back to their root, or basic form (by removing prefixes and suffixes) is known as stemming. 

The most widely used stemmer, or program/method that performs stemming, is the _Porter Stemmer_, which was originally published in 1980 by Martin Porter. An improved version was released in 2000, which fixed a number of errors. NLTK includes the Porter Stemmer, which can be used with scikit learn by creating a special function that tokenizes text documents and passing this function as an argument to the `CountVectorizer` via the `tokenizer` attribute. By performing stemming inside this tokenize method, we can return a set of tokens for a document that have been stemmed. In the following Code cell, we use a custom `tokenize` method that first builds a list of tokens by using NLTK, and then maps the Porter stemmer to the list of tokens to generate a stemmed list.

-----
[ws]: https://en.wikipedia.org/wiki/Stemming

In [10]:
import string
import nltk
from nltk.stem.porter import PorterStemmer

# Define function to tokenize text and apply stemmer
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in string.punctuation]

    stemmer = PorterStemmer()
    stems = map(stemmer.stem, tokens)
    return stems

# Define vocabulary, including stemming from our function
pclf.set_params(cv__stop_words = 'english',
                cv__ngram_range=(1,3),
                cv__lowercase=True,
                cv__tokenizer=tokenize)

# Fit, predict, and display results
pclf.fit(mvr_train, y_train)
y_pred = pclf.predict(mvr_test[0:500])
print(metrics.classification_report(y_test, y_pred, 
                                    target_names = mvr.target_names))

             precision    recall  f1-score   support

        neg       0.83      0.77      0.80       259
        pos       0.77      0.83      0.80       241

avg / total       0.80      0.80      0.80       500



In [11]:
# Extract the classifier
clf = pclf.steps[1][1]

# Display number of features
print(f'Number of Features = {clf.feature_log_prob_.shape[1]}')

Number of Features = 80529


-----

<font color='red' size = '5'> Student Exercise </font>

In the preceding cells, we incorporated the Porter Stemmer into the classification pipeline. Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Did the Porter Stemmer improve the classification results (note you need to be sure you are comparing exactly the same pipeline (including the use of ngrams)? 

2. Can you compute the number of features that the Porter Stemmer generates? How does this compare the number of features without stemming?

3. Try using a different stemming algorithm, such as [_snowball_][nsw]. How do the classification results change? 

-----
[nsw]: http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.snowball

[[Back to TOC]](#Table-of-Contents)

## Clustering Analysis

We can also apply clustering analysis to our feature matrix. While finding an unknown number of clusters in text documents can be difficult, we can learn about our data by identifying the clusters for our **known** labels. To demonstrate, in the following Code cells, we employ k-means to find two clusters in our feature matrix (the movie reviews), after which we identify the most frequently used words in each cluster.

-----

In [12]:
# Apply k-means clustering to movie reviews
from sklearn.cluster import KMeans

# Pos/Neg clusters
true_k = 2

km = KMeans(n_clusters=true_k, init='k-means++', 
            max_iter=100, n_init=1)

# Verify attributes
cv = CountVectorizer(stop_words = 'english', \
                     ngram_range=(1, 3), 
                     max_features=100000)

# Transform vocabulary
train_counts = cv.fit_transform(mvr_train)
test_data = cv.transform(mvr_test)

# Determine clusters
km.fit(test_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=2, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [13]:
# Extract top tokens
top_tokens = 20
labels = ['Neg', 'Pos']

print(f'Top {top_tokens} tokens per cluster:\n')

# Compute cluster centers
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

# Extract tokens
terms = cv.get_feature_names()

# Display top tokens per cluster
for idx in range(true_k):
    print(f'Cluster {idx}:', end='')
    for jdx in order_centroids[idx, :top_tokens]:
        print(f' {terms[jdx]}', end='')
    print('\n')

Top 20 tokens per cluster:

Cluster 0: joe mickey mallory cowboy kill new dream way new york york american like life just scene killers american dream man place clyde

Cluster 1: film movie like just time good story character way characters make does plot really scene people life bad little man



-----

We can perform the same analysis on a more complex problem by analyzing the twenty newsgroup data set. First we load the data, after which we apply k-means and identify the most common tokens in each cluster.

-----

In [14]:
# load dataset
from sklearn.datasets import fetch_20newsgroups

# Load training and testing samples
train = fetch_20newsgroups(data_home=home + 'textdm', 
                           subset='train', shuffle=True, random_state=23)
test = fetch_20newsgroups(data_home=home + 'textdm', 
                          subset='test', shuffle=True, random_state=23)

In [15]:
# Cluster twenty newsgroups
true_k = 20
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

# Verify attributes
cv = CountVectorizer(stop_words = 'english', 
                     max_features=100000)

# Transform words
train_counts = cv.fit_transform(train['data'])
test_data = cv.transform(test['data'])

# Fit clusters
km.fit(test_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=20, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [16]:
top_tokens = 20
labels = test['target']

print('Top {} tokens per cluster:\n'.format(top_tokens))

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = cv.get_feature_names()

for idx in range(true_k):
    print("Cluster {0}:".format(idx), end='')
    for jdx in order_centroids[idx, :top_tokens]:
        print(' {0}'.format(terms[jdx]), end='')
    print('\n')

Top 20 tokens per cluster:

Cluster 0: myers ms president think don dee ll know said going decision does house white today believe justice just board department

Cluster 1: jehovah elohim lord god christ father mcconkie unto son ps jesus said gods shall thou thee mormon thy earth stated

Cluster 2: planet earth spacecraft solar surface sun venus moon atmosphere planets jupiter kilometers miles years space degrees moons mars saturn mariner

Cluster 3: edu com cancer use file health ftp server disease 1993 information hiv medical number new 11 venus rx xfree86 available

Cluster 4: mb m4 ms mz ma mm mo m1 mu mc mh mw mp mt mj mk mx mf mn m3

Cluster 5: people god don just like think know say does edu way time right said did believe government new good make

Cluster 6: president mr stephanopoulos think know did general going people just don said attorney believe say decision question make time good

Cluster 7: edu writes subject article lines organization don university just com like peop

-----

<font color='red' size = '5'> Student Exercise </font>


In the preceding cells, we used k-means clustering to find clusters in text data, and to identify the most important tokens in these clusters. Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Change the vectorizer to use TF-IDF. Does this change the tokens in each cluster? 
2. Change the vectorizer to use ngrams. Does this change the tokens in each cluster?
3. Include stemming in the vectorizer. Does this change the tokens in each cluster?
4. Try using DBSCAN instead of k-means. How many clusters are found? What are the tokens in each cluster?

Finally, the k-means algorithm finds clusters. Do these clusters map directly into the _real_ categories? Feel free to discuss this on the course forums.

-----

[[Back to TOC]](#Table-of-Contents)

## Dimension Reduction

The document term matrices we have constructed in these examples can become quite large. We have already reduced the number of features used in classification problems by using stop words, by using a consistent case, and by performing stemming. On the other hand, we have enabled exponential increases in the feature space by including n-grams (although we once again restricted the feature space by using the `max_features` or the `max_df` and `min_df` attributes). The traditional dimensional reduction technique we have used in the past is PCA. However, PCA can be difficult to use with text data given the large sizes of the matrices (since a matrix inversion can be required). We can employ alternative techniques, such as incremental PCA or Truncated SVD. 

But in reality, we are less interested in finding a reduced dimensional space than we are in removing features that contain little or no information (combining features is essentially increasing the ngram range). In this case, the problem of dimension reduction becomes one of optimal feature selection. For this, we can use the scikit learn `SelectKBest` method to identify the best $k$ features. In the following Code cells, we first create a vectorizer, train it on data to demonstrate the accuracy and the number of features required to achieve that accuracy. After which, we employ `SelectKBest` to identify the best number of features, where number is predetermined, to achieve a similar accuracy. In the end, we find that less than ten percent of the original features are required to achieve the same level of accuracy.

-----

In [17]:
# Following Example was insipred by scikit learn demo
# http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html

# Apply TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')

In [18]:
# First, train on normal set of features, baseline performance.
train_counts = tf.fit_transform(train['data'])
test_data = tf.transform(test['data'])

# Naive Bayes classification
nb = MultinomialNB()
nb = nb.fit(train_counts, train['target'])
predicted = nb.predict(test_data)

# Display results
scr = 100.0 * nb.score(test_data, test['target'])
print(f'Prediction accuracy = {scr:5.1f}%')
print(f'Number of Features = {nb.feature_log_prob_.shape[1]}')

Prediction accuracy =  82.0%
Number of Features = 129791


In [19]:
# Now employ feature selection
from sklearn.feature_selection import SelectKBest, chi2

# Number of features to keep
num_k = 10000

# Employ select k best wiht chi2 since this works with sparse matrices.
ch2 = SelectKBest(chi2, k=num_k)
xtr = ch2.fit_transform(train_counts, train['target'])
xt = ch2.transform(test_data)

In [20]:
# Train simple model and compute accuracy
nb = nb.fit(xtr, train['target'])
predicted = nb.predict(xt)

# Display results
scr = 100.0 * nb.score(xt, test['target'])
print(f'Prediction accuracy = {scr:5.1f}%')
print(f'Number of Features = {nb.feature_log_prob_.shape[1]}')

Prediction accuracy =  82.0%
Number of Features = 10000


-----

We can use the feature selection to identify the top features for each category. First we extract the feature names, after which we extract the importance (or support) for each feature. By sorting the array containing the important features, we can identify the top tokens.

-----

In [21]:
# Extract feature names
feature_names = tf.get_feature_names()

# Compute most important feature names
indices = ch2.get_support(indices=True)
fn = np.array([feature_names[idx] for idx in indices])

In [22]:
# Display top 5 important feature names
top_count = 5

for idx, target in enumerate(train['target_names']):
    top_names = np.argsort(nb.coef_[idx])[-top_count:]
    tn_lst = [name for name in fn[top_names]]
    tn_lst.reverse()

    print(f'\n{target}:')
    pp.pprint(tn_lst)


alt.atheism:
['keith', 'god', 'caltech', 'atheists', 'livesey']

comp.graphics:
['graphics', 'image', 'thanks', '3d', 'files']

comp.os.ms-windows.misc:
['windows', 'dos', 'file', 'files', 'driver']

comp.sys.ibm.pc.hardware:
['drive', 'card', 'scsi', 'ide', 'bus']

comp.sys.mac.hardware:
['mac', 'apple', 'quadra', 'drive', 'centris']

comp.windows.x:
['window', 'motif', 'mit', 'com', 'server']

misc.forsale:
['sale', 'offer', 'shipping', 'distribution', 'condition']

rec.autos:
['car', 'cars', 'com', 'article', 'engine']

rec.motorcycles:
['bike', 'dod', 'com', 'ride', 'motorcycle']

rec.sport.baseball:
['baseball', 'year', 'game', 'team', 'players']

rec.sport.hockey:
['hockey', 'team', 'game', 'ca', 'nhl']

sci.crypt:
['clipper', 'key', 'encryption', 'chip', 'com']

sci.electronics:
['com', 'use', 'circuit', 'host', 'power']

sci.med:
['pitt', 'geb', 'banks', 'gordon', 'com']

sci.space:
['space', 'nasa', 'moon', 'orbit', 'henry']

soc.religion.christian:
['god', 'jesus', 'christia

In [23]:
#Clean up - removing the temporary directory we created at the beginning of the lesson
!rm -rf $HOME/temp_w9l3

-----

<font color='red' size = '5'> Student Exercise </font>

In the preceding cells, we used feature selection to identify the most important features in our simple classification pipeline. Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Change the vectorizer to change the case of all words an to employ stemming. How do the results (tokens) change?

2. Change the classification algorithm to a more accurate method. How do the results change? How does the computational time change?

Finally, what do the list of tokens say about the fact we did not remove headers or footers from the newsgroup postings? Feel free to comment on these questions in the course forum.

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. Wikipedia articles on [n-grams][wng], [Stemming][wst], and [Lemmatization][wl]
1. Google [n-gram viewer][gnv]
1. Alternative [n-gram viewer][anv]
1. Wikipedia article on [Text Clustering][wtc]
1. Blog on Sentiment Analysis with NLTK, [Part III][bsa3] and [Part IV][bsa4]
1. An online [Stemming Demo][std] using NLTK
1. A [treatise on Snowball][tsb] discussing, in depth, the process of stemming.
1. A discourse on [Dimension Reduction][msdr]
1. [Local Linear Embedding][lle], for dimensional reduction
1. [Independent Component Analysis][ica] for text data

-----

[wst]: https://en.wikipedia.org/wiki/Stemming
[wl]: https://en.wikipedia.org/wiki/Lemmatisation
[wtc]: https://en.wikipedia.org/wiki/Document_clustering

[tsb]: http://snowball.tartarus.org/texts/introduction.html
[std]: http://text-processing.com/demo/stem/

[wng]: https://en.wikipedia.org/wiki/N-gram

[gnv]: https://books.google.com/ngrams
[anv]: http://xkcd.culturomics.org

[bsa3]: http://streamhacker.com/2010/05/24/text-classification-sentiment-analysis-stopwords-collocations/
[bsa4]: http://streamhacker.com/2010/05/24/text-classification-sentiment-analysis-stopwords-collocations/

[msdr]: http://research.microsoft.com/pubs/150728/FnT_dimensionReduction.pdf
[lle]: http://science.sciencemag.org/content/290/5500/2323.abstract
[ica]: http://www.cs.rutgers.edu/~mlittman/topics/dimred02/kolenda99independent.pdf1. 

**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode